<a href="https://colab.research.google.com/github/buixuanthanh96/CS114.K21/blob/master/SacarsmDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
!git clone https://github.com/buixuanthanh96/SacarsmDetection.git

fatal: destination path 'SacarsmDetection' already exists and is not an empty directory.


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import string, re, nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def getPOS(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()#xác định loại từ
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) #trả về dưới dạng danh từ

path = "SacarsmDetection/input/"
input = pd.read_json(path + 'Sarcasm_Headlines_Dataset.json', lines=True)
input = input.drop(columns = ["article_link"])

lemmatizer = WordNetLemmatizer()
headlinesPP = []
for index, row in input.iterrows():
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.punctuation)) #Loại bỏ dấu câu
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.digits)) #Loại bỏ kí tự số
    #row["headline"] = ' '.join([word for word in row["headline"].split() if word not in (stopwords.words('english'))])#loại bỏ stopwords
    tokenList = re.sub("[^\S]", " ",  row["headline"]).split() #tách chuỗi thành các token
    #tokenList = nltk.word_tokenize(row["headline"])
    for i in range(0, len(tokenList)): #lemmatization
        tokenList[i] = lemmatizer.lemmatize(tokenList[i], getPOS(tokenList[i]))#chuyển từ về dạng nguyên mẫu
    headlinesPP.append(tokenList)
dataPP = pd.DataFrame(data = {'headline': headlinesPP, 'is_sarcastic': input["is_sarcastic"]} )
dataPP.to_csv(path+'dataPostProcessed.csv')

In [ ]:
import pandas as pd
import string, re, nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def getPOS(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()#xác định loại từ
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) #trả về dưới dạng danh từ

path = "SacarsmDetection/input/"
input = pd.read_json(path + 'new_Sarcasm_Headlines_Dataset.json', lines=True)
input = input.drop(columns = ["article_link"])

lemmatizer = WordNetLemmatizer()
headlinesPP = []
for index, row in input.iterrows():
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.punctuation)) #Loại bỏ dấu câu
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.digits)) #Loại bỏ kí tự số
    #row["headline"] = ' '.join([word for word in row["headline"].split() if word not in (stopwords.words('english'))])#loại bỏ stopwords
    tokenList = re.sub("[^\S]", " ",  row["headline"]).split() #tách chuỗi thành các token
    #tokenList = nltk.word_tokenize(row["headline"])
    for i in range(0, len(tokenList)): #lemmatization
        tokenList[i] = lemmatizer.lemmatize(tokenList[i], getPOS(tokenList[i]))#chuyển từ về dạng nguyên mẫu
    headlinesPP.append(tokenList)
dataPP = pd.DataFrame(data = {'headline': headlinesPP, 'is_sarcastic': input["is_sarcastic"]} )
dataPP.to_csv(path+'new_dataPostProcessed.csv')

ValueError: ignored

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

path = "SacarsmDetection/input/"
input = pd.read_csv(path + 'dataPostProcessed.csv', index_col=0)
X = input['headline']#.append(df['headline'])
Y = input['is_sarcastic']#.append(df['is_sarcastic'])

new_input = pd.read_csv(path + 'new_dataPostProcessed.csv', index_col=0)
X_new_test = new_input['headline']#.append(df['headline'])
Y_new_test = new_input['is_sarcastic']#.append(df['is_sarcastic'])



xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.1)
print("*** DATASET PARTITIONED IN TRAIN: "+str(len(xTrain))+ " TEST: "+str(len(xTest)))
classifiers = [
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', LinearSVC())]),
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', MLPClassifier(hidden_layer_sizes=(90,90), random_state=42, solver='lbfgs'))]),
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', RandomForestClassifier())]),
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', KNeighborsClassifier(n_neighbors=2))]),
    ]
    
for classifier in classifiers:
    classifier.fit(xTrain, yTrain)
    print("###### " + classifier.named_steps.get("classifier").__class__.__name__ + " ############")
    # print("\nTraining: " + str(classifier.score(xTrain, yTrain).round(3)))
    # print("Test: " + str(classifier.score(xTest, yTest).round(3)) +"\n")
    prediction = classifier.predict(xTest)
    prediction_new = classifier.predict(X_new_test)
    #print(confusion_matrix(yTest, prediction))
    #print(classification_report(yTest, prediction))
    print("accuracy out: ",metrics.accuracy_score(yTest, prediction))
    print("accuracy new: ",metrics.accuracy_score(Y_new_test, prediction_new))
    prediction1 = classifier.predict( ['courtroom sketch artist clear manga influence'])
    print("courtroom sketch artist clear manga influence: ",prediction1)



FileNotFoundError: ignored

In [ ]:

new_input = pd.read_csv(path + 'dataPostProcessed.csv', index_col=0)

from sklearn.model_selection import train_test_split
X_new_test = input['headline']#.append(df['headline'])
Y_new_test = input['is_sarcastic']#.append(df['is_sarcastic'])

xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.1)
print("*** DATASET PARTITIONED IN TRAIN: "+str(len(xTrain))+ " TEST: "+str(len(xTest)))
classifiers = [
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', LinearSVC())]),
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', MLPClassifier(hidden_layer_sizes=(90,90), random_state=42, solver='lbfgs'))]),
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', RandomForestClassifier())]),
    Pipeline([('tfidf', TfidfVectorizer()), ('classifier', KNeighborsClassifier(n_neighbors=2))]),
    ]
    
for classifier in classifiers:
    classifier.fit(xTrain, yTrain)
    print("###### " + classifier.named_steps.get("classifier").__class__.__name__ + " ############")
    # print("\nTraining: " + str(classifier.score(xTrain, yTrain).round(3)))
    # print("Test: " + str(classifier.score(xTest, yTest).round(3)) +"\n")
    prediction = classifier.predict(xTest)
    #print(confusion_matrix(yTest, prediction))
    #print(classification_report(yTest, prediction))
    print("accuracy: ",metrics.accuracy_score(yTest, prediction))
    prediction1 = classifier.predict( ['courtroom sketch artist clear manga influence'])
    print("courtroom sketch artist clear manga influence: ",prediction1)

